# Neural Network with LDA
MSc in Statistical Science\
University of Oxford\
Group-assessed practical\
HT 2024

这个也太差了吧！！！

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
# The scikit-learn version is 1.2.2.

The scikit-learn version is 1.2.2.


## Load dataset

In [2]:
# Load the training data and the test inputs
X_train = pd.read_csv('X_train.csv', index_col = 0, header=[0, 1, 2]) # inputs of the training set
y_train = pd.read_csv('y_train.csv', index_col=0).squeeze('columns').to_numpy() # outputs of the training set
X_test = pd.read_csv('X_test.csv', index_col = 0, header=[0, 1, 2]) # inputs of the test set

In [3]:
# X_train is a 6,000 * 518 dataframe. 
# Entries (i,j) correspond to the j'th dimension of the observation i
X_train


feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
Id                                                                         
0            -0.266585 -0.984668 -0.729823 -0.895122  2.138628  0.935209   
1            -0.180061  0.260884 -0.069373  0.208734 -0.078855 -0.577818   
2            -0.692900  0.356662  0.062617  0.248280  3.470037  0.166613   
3             0.243339  0.214182 -0.049026  1.456255 -0.360826 -0.875256   
4            -0.968576  0.309255  0.223164  0.160960  0.919838 -0.111985   
...                ...       ...       ...       ...       ...       ...   
5995          0.503490 -0.540720 -0.690117 -0.107338 -0.647856 -0.681969   
5996         -0.600597  0.406386 -0.748409 -0.316157 -0.507428 -0.054214   
5997         -1.014298 -0.950744  0.618304  0.204298 -0.788411 -0.794254   
5998         -0.002938  0.646034 -0.732819  1.205990 -0.898733 -0.684953   
5999         -0.881917 -0.615638  0.891155  4.850262 -0.349280 -1.010980   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
Id                                                  ...                       
0           0.104089 -0.698659 -0.736408 -0.334376  ...  0.065003  0.016522   
1           0.583788  0.143781  0.291556  0.007314  ...  0.087692  0.016355   
2           0.823874  0.181112  0.551939  0.357985  ...  0.132387  0.025847   
3          -0.770200  0.315500  0.789956  0.448319  ...  0.071478  0.019166   
4          -1.012521 -0.665692 -0.316646 -0.264381  ...  0.106220  0.023536   
...              ...       ...       ...       ...  ...       ...       ...   
5995       -0.246245 -0.546552  0.062783  0.070393  ...  0.084929  0.017250   
5996       -0.476804 -0.373120 -0.930158 -1.080690  ...  0.075407  0.014998   
5997       -0.586847  0.099172 -0.313476 -0.523417  ...  0.138591  0.024969   
5998        0.134642 -0.374792 -0.019524 -1.016032  ...  0.137695  0.030371   
5999       -0.577905 -0.046604 -0.828026 -0.457468  ...  0.175120  0.039703   

feature                      zcr                                          \
statistics              kurtosis       max      mean    median       min   
number            06          01        01        01        01        01   
Id                                                                         
0           0.015776    5.743597  0.307617  0.051370  0.042480  0.002441   
1           0.016605   64.870987  0.812988  0.082784  0.069824  0.003906   
2           0.023922   34.251705  0.850098  0.058200  0.036621  0.010254   
3           0.025535    1.364990  0.342285  0.081713  0.075195  0.000000   
4           0.019742    3.589230  0.322266  0.073736  0.069336  0.004395   
...              ...         ...       ...       ...       ...       ...   
5995        0.020335    4.868783  0.668945  0.076452  0.044434  0.001465   
5996        0.020683    7.893681  0.584961  0.076210  0.048340  0.000000   
5997        0.023658   27.257378  0.373047  0.042598  0.037598  0.000000   
5998        0.029970  431.200500  0.384277  0.025731  0.025391  0.008301   
5999        0.025094   13.567758  0.233398  0.047689  0.044922  0.015625   

feature                          
statistics       skew       std  
number             01        01  
Id                               
0            1.976972  0.034533  
1            7.374503  0.074870  
2            5.927942  0.117603  
3            1.100437  0.041754  
4            1.210593  0.036459  
...               ...       ...  
5995         2.045856  0.084214  
5996         2.561808  0.073010  
5997         3.778109  0.027813  
5998        10.260160  0.006870  
5999         2.553229  0.018049  

[6000 rows x 518 colu

In [4]:
#y_train contains the true class:  Electronic, Experimental, Folk, Hip-Hop, Instrumental, International, Pop or Rock
y_train

array(['Electronic', 'Rock', 'Instrumental', ..., 'Pop', 'Instrumental',
       'Instrumental'], dtype=object)

In [5]:
# X_test is the array of test inputs, of the same format as X_train. The objective is to predict the class (Electronic, Experimental, Folk, Hip-Hop, Instrumental, International, Pop or Rock) of the output
X_test

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
Id                                                                         
0             3.974762  4.354650  3.394523  0.033462  5.611623  2.592655   
1             0.033636 -0.139950  0.678688 -0.553824 -0.165293  0.370275   
2             0.044094  4.285359  0.977213  3.240997  0.400350  1.026224   
3            -0.425218  0.793007  0.509624 -0.967103 -1.432252 -0.900761   
4            -1.120238 -0.503659  0.303515 -0.596549 -0.716761 -0.874363   
...                ...       ...       ...       ...       ...       ...   
1995         -0.166039 -0.660406 -0.955245 -0.689303 -0.916851 -0.136661   
1996          2.010343  0.344572  2.738141  1.238262  2.815471  0.689118   
1997         -0.869148 -0.600280  0.105814  0.553810 -0.839182 -0.706434   
1998         -0.854251 -0.804227  1.347275 -0.748155 -0.408385 -0.909165   
1999          0.929528  0.922116  1.074849  0.431512  0.040441 -0.361733   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
Id                                                  ...                       
0           3.041094  2.815378  3.954026  2.365586  ...  0.052970  0.013487   
1          -0.314710 -0.368706 -0.437181 -0.441662  ...  0.093105  0.022865   
2           0.772464  0.807625  1.942534  1.938970  ...  0.081955  0.017371   
3          -0.501279 -0.855886 -0.556825  7.404243  ...  0.098877  0.019089   
4          -0.708101 -0.642351 -0.327327 -0.342220  ...  0.102545  0.023823   
...              ...       ...       ...       ...  ...       ...       ...   
1995       -0.367279 -1.135291 -1.174082 -0.609749  ...  0.060857  0.022777   
1996        4.040531  2.749969  2.658481  4.763800  ...  0.050589  0.015169   
1997       -0.360566  0.053638 -0.791513 -0.669329  ...  0.079556  0.025229   
1998       -0.870467 -0.077660 -0.538250 -0.108390  ...  0.147899  0.023989   
1999        0.371801  0.470401 -0.566806  0.082485  ...  0.119940  0.023605   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
Id                                                                        
0           0.022031  70.844788  0.671387  0.035129  0.026367  0.010254   
1           0.028800  18.427612  0.538574  0.055975  0.039062  0.000000   
2           0.016673  38.401405  0.405762  0.030685  0.028809  0.002930   
3           0.016183   4.129582  0.252441  0.037955  0.024902  0.000000   
4           0.025166  16.758356  0.380371  0.027851  0.019043  0.000000   
...              ...        ...       ...       ...       ...       ...   
1995        0.017722   8.467750  0.444824  0.055770  0.039062  0.001465   
1996        0.013096  -0.649581  0.357422  0.126850  0.120117  0.000000   
1997        0.026858   1.563682  0.152344  0.039138  0.035156  0.002441   
1998        0.027429  22.050222  0.319824  0.032715  0.028809  0.000000   
1999        0.024036   2.444547  0.256836  0.047209  0.041016  0.002441   

feature                         
statistics      skew       std  
number            01        01  
Id                              
0           8.394708  0.067026  
1           3.638194  0.053879  
2           2.620369  0.016835  
3           1.898847  0.034382  
4           3.244483  0.027128  
...              ...       ...  
1995        2.520271  0.056601  
1996        0.234551  0.055476  
1997        1.084503  0.020284  
1998        3.429668  0.022654  
1999        1.266626  0.030202  

[2000 rows x 518 columns]

In [3]:
X = X_train
y = y_train

# Split the data into training and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [4]:
# Scale the features using StandardScaler
scalerx = StandardScaler()
X_train = scalerx.fit_transform(X_train)
X_val = scalerx.transform(X_val)

In [5]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [6]:
# Apply LDA for dimensionality reduction
lda = LinearDiscriminantAnalysis(n_components=7)
X_train_lda = lda.fit_transform(X_train, y_train)
X_val_lda = lda.transform(X_val)

In [7]:
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)

In [8]:
X_train_lda.shape

(4800, 7)

## Classification with NN and prediction on the test set

In [36]:
# Define the model

model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), kernel_initializer='he_normal', activation='tanh'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='tanh', kernel_initializer='he_normal'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu',))
model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(16, activation='relu', ))
model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))  # number of classes for multi-class

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping based on validation accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the validation set
eval_result = model.evaluate(X_val, y_val)
print(f"Validation Loss: {eval_result[0]}, Validation Accuracy: {eval_result[1]}")


Epoch 1/500
108/108 [==============================] - 4s 9ms/step - loss: 2.0048 - accuracy: 0.2335 - val_loss: 1.7686 - val_accuracy: 0.3620
Epoch 2/500
108/108 [==============================] - 1s 6ms/step - loss: 1.6892 - accuracy: 0.3727 - val_loss: 1.5570 - val_accuracy: 0.4661
Epoch 3/500
108/108 [==============================] - 1s 6ms/step - loss: 1.4956 - accuracy: 0.4511 - val_loss: 1.5083 - val_accuracy: 0.4714
Epoch 4/500
108/108 [==============================] - 1s 5ms/step - loss: 1.3797 - accuracy: 0.5075 - val_loss: 1.4291 - val_accuracy: 0.4974
Epoch 5/500
108/108 [==============================] - 1s 8ms/step - loss: 1.2721 - accuracy: 0.5382 - val_loss: 1.3792 - val_accuracy: 0.5182
Epoch 6/500
108/108 [==============================] - 1s 8ms/step - loss: 1.1910 - accuracy: 0.5689 - val_loss: 1.3562 - val_accuracy: 0.5495
Epoch 7/500
108/108 [==============================] - 1s 9ms/step - loss: 1.1079 - accuracy: 0.6065 - val_loss: 1.3640 - val_accuracy: 0.5521

In [11]:
from kerastuner import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation=hp.Choice('activation', values=['relu', 'softplus', 'tanh', 'sigmoid']),
                    input_shape=(7,),
                    kernel_initializer=hp.Choice('kernel_initializer', values=['glorot_normal', 'he_normal', 
                                                                               'random_normal', 'glorot_normal', 'random_uniform'])
                    ))
    model.add(Dropout(hp.Float('dropout_input', min_value=0.0, max_value=0.7, step=0.1)))
    for i in range(hp.Int('layers', 1, 5)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                        activation=hp.Choice(f'activation_{i}', values=['relu', 'softplus','tanh', 'sigmoid']),
                        kernel_initializer=hp.Choice(f'kernel_initializer_{i}', values=['glorot_normal', 'he_normal',
                                                                                        'random_normal', 'glorot_normal', 'random_uniform']),
                        ))
        if hp.Choice(f'add_dropout_{i}', values=[True, False]):
            model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.7, step=0.1)))
    
    model.add(Dense(8, activation='softmax'))
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Set smaller during experimentation
    executions_per_trial=1,  # Increase for more robust results
    directory='my_dir_ldann',
    project_name='hparam_tuning'
)

tuner.search(X_train_lda, y_train_cat, epochs=10, validation_data=(X_val_lda, y_val_cat))  # Use a lower number of epochs for the search phase



Trial 5 Complete [00h 00m 15s]
val_accuracy: 0.5666666626930237

Best val_accuracy So Far: 0.5666666626930237
Total elapsed time: 00h 01m 10s


In [12]:
tuner.results_summary()

# Retrieve the best hyperparameters
best_hps = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print("Best Hyperparameters:")
print(best_hps)

Results summary
Results in my_dir_ldann\hparam_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
units: 448
activation: relu
kernel_initializer: glorot_normal
dropout_input: 0.6000000000000001
layers: 3
units_0: 224
activation_0: relu
kernel_initializer_0: random_normal
add_dropout_0: 0
dropout_0: 0.4
units_1: 384
activation_1: relu
kernel_initializer_1: glorot_normal
add_dropout_1: 1
dropout_1: 0.30000000000000004
units_2: 96
activation_2: relu
kernel_initializer_2: glorot_normal
add_dropout_2: 0
dropout_2: 0.6000000000000001
units_3: 352
activation_3: relu
kernel_initializer_3: random_uniform
add_dropout_3: 1
dropout_3: 0.6000000000000001
units_4: 448
activation_4: tanh
kernel_initializer_4: glorot_normal
add_dropout_4: 1
dropout_4: 0.0
Score: 0.5666666626930237

Trial 1 summary
Hyperparameters:
units: 128
activation: sigmoid
kernel_initializer: glorot_normal
dropout_input: 0.4
layers: 2
units_0: 128
activation_0: relu
ker

In [13]:
# Get the best hyperparameters
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_hps = best_trial.hyperparameters.values
print("Best Hyperparameters:")
print(best_hps)

# Build the best model using the retrieved hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model on the full training dataset
best_model.fit(X_train_lda, y_train_cat, epochs=10, batch_size=32, validation_data=(X_val_lda, y_val_cat))

# Evaluate the model on the test set (assuming you have a separate test set)
eval_result = best_model.evaluate(X_val_lda, y_val_cat)
print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")


Best Hyperparameters:
{'units': 448, 'activation': 'relu', 'kernel_initializer': 'glorot_normal', 'dropout_input': 0.6000000000000001, 'layers': 3, 'units_0': 224, 'activation_0': 'relu', 'kernel_initializer_0': 'random_normal', 'add_dropout_0': 0, 'dropout_0': 0.4, 'units_1': 384, 'activation_1': 'relu', 'kernel_initializer_1': 'glorot_normal', 'add_dropout_1': 1, 'dropout_1': 0.30000000000000004, 'units_2': 96, 'activation_2': 'relu', 'kernel_initializer_2': 'glorot_normal', 'add_dropout_2': 0, 'dropout_2': 0.6000000000000001, 'units_3': 352, 'activation_3': 'relu', 'kernel_initializer_3': 'random_uniform', 'add_dropout_3': 1, 'dropout_3': 0.6000000000000001, 'units_4': 448, 'activation_4': 'tanh', 'kernel_initializer_4': 'glorot_normal', 'add_dropout_4': 1, 'dropout_4': 0.0}
Epoch 1/10
150/150 [==============================] - 3s 10ms/step - loss: 0.8796 - accuracy: 0.6963 - val_loss: 1.4802 - val_accuracy: 0.5483
Epoch 2/10
150/150 [==============================] - 1s 8ms/step - 

In [14]:
X_test = scalerx.transform(X_test)
X_test_lda = lda.transform(X_test)

In [15]:
predictions = best_model.predict(X_test_lda) # compute predictions on the test inputs
predictions

63/63 [==============================] - 0s 4ms/step


array([[9.7691226e-01, 5.1736915e-05, 2.2135160e-04, ..., 4.1883904e-06,
        1.9446468e-03, 1.6711859e-02],
       [4.5093424e-03, 4.4833388e-02, 6.7932080e-03, ..., 4.2576477e-02,
        8.0725753e-01, 4.6822593e-02],
       [5.6053413e-04, 1.4025713e-03, 1.5357587e-03, ..., 9.9267358e-01,
        2.8285622e-03, 4.5467934e-04],
       ...,
       [6.9615409e-02, 1.5663022e-01, 5.3764634e-02, ..., 3.1271454e-02,
        9.2513770e-02, 3.7443283e-01],
       [2.1055047e-03, 3.0909274e-02, 2.8312027e-02, ..., 1.9821497e-02,
        6.3567829e-01, 2.6380521e-01],
       [2.1930754e-01, 1.4384274e-01, 2.0180966e-03, ..., 1.9662710e-02,
        5.6056535e-01, 6.3193371e-03]], dtype=float32)

In [16]:
predicted_labels = np.argmax(predictions, axis=1)   # Adding 1 to make labels 1-indexed

# Display the predicted class labels
print("Predicted Class Labels:")
print(predicted_labels)

Predicted Class Labels:
[0 6 5 ... 7 6 6]


In [17]:
y_pred_dec = label_encoder.inverse_transform(predicted_labels)
y_pred_dec

array(['Electronic', 'Pop', 'International', ..., 'Rock', 'Pop', 'Pop'],
      dtype=object)

## Export in csv format 

In [31]:
# Export the predictions on the test data in csv format
prediction = pd.DataFrame(y_pred, columns=['Genre'])
prediction.index.name='Id'
prediction.to_csv('myprediction.csv') # export to csv file

# The csv file should be of the form
#Id, Genre
#0, Folk
#1, Hip-Hop
#2, International
#...
#1998, Experimental
#1999, Pop